In [225]:
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score   
import h5py
from keras import layers
import numpy as np
import matplotlib as plt

In [226]:
VALID_SPLIT = 0.2
TEST_SPLIT = 0.2

### Read data backout

In [227]:
def read_hdf(file_path):
    return_dict = []
    with h5py.File(file_path, 'r') as hf:
        print(list(hf.keys()))
        print(file_path[9:-5])
        dataset = hf[file_path[9:-5]]
        print(len(list(dataset.keys())))
        keys = list(dataset.keys())
        for i in keys:
            return_dict.append({
                "name":i.replace("-","/"),
                "data":dataset[i][:]
            })
        return return_dict

In [228]:
genre = ['Experimental', 'Electronic', 'Rock', 'Instrumental', 'Pop', 'Folk', 'Hip-Hop', 'International']

In [229]:
#!unzip mel_spec/update_feature.zip
# Move all files into mel_sepc then

In [230]:
hdf5_dict["Experimental"] = read_hdf("mel_spec/Experimental.hdf5")

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'mel_spec/Experimental.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [231]:
mel_spec_base_path = "mel_spec/"
for i in genre:
    output = read_hdf(mel_spec_base_path+i+".hdf5") # list of dictionaries
    hdf5_dict[i] = output


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'mel_spec/Experimental.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [232]:
hdf5_dict["International"][0]["data"]

array([[-38.476818, -40.88924 , -41.310997, ..., -42.49931 , -43.811142,
        -46.01673 ],
       [-37.620705, -42.210247, -43.371998, ..., -34.7596  , -36.702255,
        -37.005035],
       [-29.18562 , -25.22186 , -26.829031, ..., -31.940666, -28.140745,
        -28.65306 ],
       ...,
       [-80.      , -80.      , -80.      , ..., -80.      , -80.      ,
        -80.      ],
       [-80.      , -80.      , -80.      , ..., -80.      , -80.      ,
        -80.      ],
       [-80.      , -80.      , -80.      , ..., -80.      , -80.      ,
        -80.      ]], dtype=float32)

In [233]:
label_reference = {}
for index,value in enumerate(hdf5_dict.keys()):
    label_reference[value] = index

In [234]:
label_reference
print(list(label_reference.values()))

[0, 1, 2, 3, 4, 5, 6, 7]


In [235]:
def splitting_test_valid(dictionary,test_size,valid_size):
    x = []
    y = []
    for genres,datas in dictionary.items():
        for i in datas:
            x.append(i['data'])
            y.append(label_reference[genres])
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = test_size)
    x_train,x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size = valid_size)
    return x_train,x_valid,x_test,y_train,y_valid,y_test

In [236]:
x_train,x_valid,x_test,y_train,y_valid,y_test = splitting_test_valid(hdf5_dict,0.2,0.2)

In [237]:

x_train = np.rollaxis(np.dstack(x_train),-1)
x_valid = np.rollaxis(np.dstack(x_valid),-1)
x_test = np.rollaxis(np.dstack(x_test),-1)
y_train = np.rollaxis(np.dstack(y_train),-1)
y_valid = np.rollaxis(np.dstack(y_valid),-1)
y_test = np.rollaxis(np.dstack(y_test),-1)


In [238]:
x_train = x_train[...,np.newaxis]
x_valid = x_valid[...,np.newaxis]
x_test = x_test[...,np.newaxis]

In [239]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(5116, 128, 646, 1)
(1279, 128, 646, 1)
(1599, 128, 646, 1)
(5116, 1, 1)
(1279, 1, 1)
(1599, 1, 1)


In [240]:
x_train[0].shape

(128, 646, 1)

### Model 

In [246]:
'''
Since the task is not overly ccomplex, choose to use LeNet 5
'''
input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
# inputs = keras.Input(shape=input_shape) 

def model(input_shape,label_reference):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=input_shape))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(len(label_reference), activation='softmax'))
    return model



In [247]:
model = model(input_shape,label_reference)

In [244]:
optimizer = keras.optimizers.SGD(learning_rate = 0.0009)
model.compile(optimizer = optimizer, loss = "SparseCategoricalCrossentropy",metrics = ["accuracy"])

### Fitting and training the data

In [245]:
model.fit(x_train,y_train,validation_data = (x_valid,y_valid),epochs=30,batch_size = 4)

Epoch 1/30
1279/1279 [==============================] - 119s 92ms/step - loss: 2.6216 - accuracy: 0.1857 - val_loss: 2.3576 - val_accuracy: 0.2213
Epoch 2/30
1279/1279 [==============================] - 99s 77ms/step - loss: 2.3282 - accuracy: 0.2336 - val_loss: 2.5467 - val_accuracy: 0.2283
Epoch 3/30
1279/1279 [==============================] - 87s 68ms/step - loss: 2.1948 - accuracy: 0.2508 - val_loss: 1.8138 - val_accuracy: 0.3378
Epoch 4/30
1279/1279 [==============================] - 76s 59ms/step - loss: 2.0837 - accuracy: 0.2813 - val_loss: 1.6287 - val_accuracy: 0.3948
Epoch 5/30
1279/1279 [==============================] - 89s 70ms/step - loss: 1.9795 - accuracy: 0.2875 - val_loss: 1.8499 - val_accuracy: 0.3393
Epoch 6/30
1279/1279 [==============================] - 89s 69ms/step - loss: 1.8868 - accuracy: 0.3176 - val_loss: 1.7116 - val_accuracy: 0.3862
Epoch 7/30
1279/1279 [==============================] - 85s 66ms/step - loss: 1.8225 - accuracy: 0.3434 - val_loss: 1.6459 

### Checking the result

In [202]:
error,accuracy = model.evaluate(x_test[0:4],y_test[0:4],verbose = 1)
print(f"Error is {error} and accuracy is {accuracy}")

1/1 [==============================] - 0s 142ms/step - loss: 2.7212 - accuracy: 0.2500
Error is 2.721205711364746 and accuracy is 0.25


In [159]:
def predict_list(test_list):
    prediction = model.predict(test_list)
    prediction = [np.argmax(i) for i in prediction]
    return prediction

In [153]:
print(y_test.shape)

list_label = list(y_test.flatten())
print(len(list_label))

(1599, 1, 1)
1599


In [248]:
from sklearn.metrics import classification_report

In [249]:
prediction_class = predict_list(x_test)
real_class = list(y_test.flatten())


50/50 [==============================] - 6s 114ms/step


In [250]:
precision = precision_score(real_class,prediction_class,average="macro")
recall = recall_score(real_class,prediction_class,average="macro")
f1_score = f1_score(real_class,prediction_class,average="macro")
accuracy = accuracy_score(real_class, prediction_class)  

/Users/peterhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [251]:
print(confusion_matrix(real_class, prediction_class,labels = list(label_reference.values())))

[[  0   0   0   0   4 199   0   0]
 [  0   0   0   0   4 198   0   0]
 [  0   0   0   0   2 199   0   0]
 [  1   0   0   0   5 184   0   0]
 [  0   0   0   0   7 209   0   0]
 [  0   0   0   0   3 198   0   0]
 [  0   0   0   0   2 189   0   0]
 [  0   0   0   0   2 193   0   0]]


In [252]:

print(f"Accuracy score is {accuracy}")
print(f"Precision score is {precision}")
print(f"Recall score is {recall}")
print(f"F1 score is {f1_score}")


Accuracy score is 0.1282051282051282
Precision score is 0.045946792378189494
Recall score is 0.12718525428413488
F1 score is 0.035108958837772396


In [253]:
print(classification_report(real_class, prediction_class))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       203
           1       0.00      0.00      0.00       202
           2       0.00      0.00      0.00       201
           3       0.00      0.00      0.00       190
           4       0.24      0.03      0.06       216
           5       0.13      0.99      0.22       201
           6       0.00      0.00      0.00       191
           7       0.00      0.00      0.00       195

    accuracy                           0.13      1599
   macro avg       0.05      0.13      0.04      1599
weighted avg       0.05      0.13      0.04      1599



/Users/peterhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/peterhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/peterhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
